# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
ls event_data/*

event_data/2018-11-01-events.csv  event_data/2018-11-16-events.csv
event_data/2018-11-02-events.csv  event_data/2018-11-17-events.csv
event_data/2018-11-03-events.csv  event_data/2018-11-18-events.csv
event_data/2018-11-04-events.csv  event_data/2018-11-19-events.csv
event_data/2018-11-05-events.csv  event_data/2018-11-20-events.csv
event_data/2018-11-06-events.csv  event_data/2018-11-21-events.csv
event_data/2018-11-07-events.csv  event_data/2018-11-22-events.csv
event_data/2018-11-08-events.csv  event_data/2018-11-23-events.csv
event_data/2018-11-09-events.csv  event_data/2018-11-24-events.csv
event_data/2018-11-10-events.csv  event_data/2018-11-25-events.csv
event_data/2018-11-11-events.csv  event_data/2018-11-26-events.csv
event_data/2018-11-12-events.csv  event_data/2018-11-27-events.csv
event_data/2018-11-13-events.csv  event_data/2018-11-28-events.csv
event_data/2018-11-14-events.csv  event_data/2018-11-29-events.csv
event_data/2018-11-15-events.csv  event_data/2018-11-30-events

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
full_data_rows_list[1]

['Jimi Hendrix',
 'Logged In',
 'Mohammad',
 'M',
 '1',
 'Rodriguez',
 '239.82975',
 'paid',
 'Sacramento--Roseville--Arden-Arcade, CA',
 'PUT',
 'NextSong',
 '1.54051E+12',
 '961',
 'Woodstock Inprovisation',
 '200',
 '1.54328E+12',
 '88']

In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()
print("connection success ")

connection success 


#### Create Keyspace

In [8]:
try:
        session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
        print('Keyspace Created')
except Exception as e:
        print(e)

Keyspace Created


#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Drop & Create session table to retrive data for 1st Query 

In [10]:
drop_query1 = "DROP TABLE IF EXISTS songs_sessions"

try:
    session.execute(drop_query1)
except Exception as e:
    print(e) 


create_query1 = """
create table if not exists songs_sessions (
            session_id int,
            item_in_session int,
            artist text,
            song text,
            length float,
            PRIMARY KEY (session_id, item_in_session)
            )"""

try:
    session.execute(create_query1)
    print("Table Created")
except Exception as e:
    print(e)          

Table Created


### Insert all data from the csv files into the songs_sessions table

In [11]:
df_chunks = pd.read_csv('event_datafile_new.csv',chunksize=1000)

insert_query1 = "INSERT INTO songs_sessions (session_id, item_in_session , artist, song,length)"
insert_query1  = insert_query1 + " VALUES (%s, %s, %s, %s, %s)"

for i,df_chunks in enumerate(df_chunks):
    print(f'Chnuk rows of {i}: {len(df_chunks)}')
    print(f'Chnuk memory usage: {df_chunks.memory_usage(deep=True).sum()*1e-6} MB')
    for index, row in df_chunks.iterrows():
        try:
            session.execute(insert_query1,
                            (row.sessionId,
                             row.itemInSession,
                             row.artist,
                             row.song,
                             row.length)
                           )
        
        except Exception as e:
            print(e)

Chnuk rows of 0: 1000
Chnuk memory usage: 0.516069 MB
Chnuk rows of 1: 1000
Chnuk memory usage: 0.5145959999999999 MB
Chnuk rows of 2: 1000
Chnuk memory usage: 0.51791 MB
Chnuk rows of 3: 1000
Chnuk memory usage: 0.5153829999999999 MB
Chnuk rows of 4: 1000
Chnuk memory usage: 0.515973 MB
Chnuk rows of 5: 1000
Chnuk memory usage: 0.515053 MB
Chnuk rows of 6: 820
Chnuk memory usage: 0.422612 MB


#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
select_query1 = """
SELECT 
    artist, song, length 
FROM 
    songs_sessions 
WHERE 
    session_id = 338 
AND
    item_in_session = 4
"""
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Drop & Create session table to retrive data for 2nd Query

In [13]:
drop_query2 = "DROP TABLE IF EXISTS users_sessions"

try:
    session.execute(drop_query2)
except Exception as e:
    print(e) 


create_query2 = """
create table if not exists users_sessions (
            session_id int,
            user_id int,
            artist text,
            song text,
            first_Name text,
            last_Name text,
            item_in_session int,
            PRIMARY KEY ((user_id, session_id), item_in_session)
            )"""

try:
    session.execute(create_query2)
    print("Table Created")
except Exception as e:
    print(e)   

Table Created


### Read and insert data using Pandas Dataframe

In [14]:
df_chunks2 = pd.read_csv('event_datafile_new.csv',chunksize=1000)

insert_query2 = "INSERT INTO users_sessions (artist,first_Name,item_in_session,last_Name,session_id,song,user_id)"
insert_query2  = insert_query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

for i,df_chunks2 in enumerate(df_chunks2):
    print(f'Chnuk rows of {i}: {len(df_chunks2)}')
    print(f'Chnuk memory usage: {df_chunks2.memory_usage(deep=True).sum()*1e-6} MB')
    for index, row in df_chunks2.iterrows():
        try:
            session.execute(insert_query2,
                            (row.artist,
                             row.firstName,
                             row.itemInSession,
                             row.lastName,
                             row.sessionId,
                             row.song,
                             row.userId)
                           )
        except Exception as e:
            print(e)

Chnuk rows of 0: 1000
Chnuk memory usage: 0.516069 MB
Chnuk rows of 1: 1000
Chnuk memory usage: 0.5145959999999999 MB
Chnuk rows of 2: 1000
Chnuk memory usage: 0.51791 MB
Chnuk rows of 3: 1000
Chnuk memory usage: 0.5153829999999999 MB
Chnuk rows of 4: 1000
Chnuk memory usage: 0.515973 MB
Chnuk rows of 5: 1000
Chnuk memory usage: 0.515053 MB
Chnuk rows of 6: 820
Chnuk memory usage: 0.422612 MB


In [15]:
select_query2 = """
SELECT 
    artist, song, first_Name, last_Name 
FROM 
    users_sessions 
WHERE 
    user_id = 10 
AND
    session_id = 182
"""
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### Drop & Create session table to retrive data for 3rd Query

In [16]:
drop_query3 = "DROP TABLE IF EXISTS users_songs"

try:
    session.execute(drop_query3)
except Exception as e:
    print(e) 


create_query3 = ("""
create table if not exists users_songs (
            song text,
            user_Id int,
            first_Name text,
            last_Name text,
            PRIMARY KEY(song, user_Id))
            """)

try:
    session.execute(create_query3)
    print("Table Created")
except Exception as e:
    print(e)

Table Created


In [ ]:
df_chunks3 = pd.read_csv('event_datafile_new.csv',chunksize=1000)
insert_query3 = "INSERT INTO users_songs (song, user_Id, first_Name, last_Name)"
insert_query3  = insert_query3 + " VALUES (%s, %s, %s, %s)"

for i,df_chunks3 in enumerate(df_chunks3):
    print(f'Chnuk rows of {i}: {len(df_chunks3)}')
    print(f'Chnuk memory usage: {df_chunks3.memory_usage(deep=True).sum()*1e-6} MB')
    for index, row in df_chunks3.iterrows():
        try:
            session.execute(insert_query3,
                            (row.song,
                             row.userId,
                             row.firstName,
                             row.lastName)
                           )
        except Exception as e:
            print(e)

Chnuk rows of 0: 1000
Chnuk memory usage: 0.516069 MB
Chnuk rows of 1: 1000
Chnuk memory usage: 0.5145959999999999 MB
Chnuk rows of 2: 1000
Chnuk memory usage: 0.51791 MB
Chnuk rows of 3: 1000
Chnuk memory usage: 0.5153829999999999 MB
Chnuk rows of 4: 1000
Chnuk memory usage: 0.515973 MB
Chnuk rows of 5: 1000
Chnuk memory usage: 0.515053 MB
Chnuk rows of 6: 820
Chnuk memory usage: 0.422612 MB


In [57]:
select_query3 = """
SELECT 
    song,
    first_Name,
    last_Name 
FROM 
    users_songs 
WHERE 
    song = 'All Hands Against His Own' 
"""
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(song='All Hands Against His Own', first_name='Jacqueline', last_name='Lynch')
Row(song='All Hands Against His Own', first_name='Tegan', last_name='Levine')
Row(song='All Hands Against His Own', first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [59]:
session.execute("DROP TABLE IF EXISTS songs_sessions")
session.execute("DROP TABLE IF EXISTS users_sessions")
session.execute("DROP TABLE IF EXISTS users_songs")

### Close the session and cluster connection¶

In [60]:
session.shutdown()
cluster.shutdown()